Importing the necessary libraries:

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn import cluster
from PIL import Image

Loading and testing an image via Pillow Library

In [2]:
#A variable is assigned to open and hold the test image
imageTest = Image.open("imageTest.jpg");

#some details checking about the image
print(imageTest.format);
print(imageTest.size);
print(imageTest.mode);

JPEG
(1360, 768)
RGB


Converting the image loaded into an Array from NumPy library

In [3]:
#using np.array() to convert an image in numpy array
imageArray = np.array(imageTest);

#printing the typr of it
print(type(imageArray));

#printing the shape of the image
print(imageArray.shape);

#priting the data of the imageArray representing each pixel of it
print(imageArray);

#Creating an image from the imageArray
imageCreated = Image.fromarray(imageArray);
print(type(imageCreated));

#Check the image details
print(imageCreated.mode);
print(imageCreated.size);

#converting the image to greyscale
imageGreyscale = imageTest.convert("L");

#saving the image
imageGreyscale.save("greyscale.jpeg");

#Priting the data of the imageGreyscale
print(np.array(imageGreyscale));

<class 'numpy.ndarray'>
(768, 1360, 3)
[[[ 75   2  73]
  [ 75   2  73]
  [ 75   2  73]
  ...
  [ 76   3  72]
  [ 75   2  71]
  [ 75   2  71]]

 [[ 75   2  73]
  [ 75   2  73]
  [ 75   2  73]
  ...
  [ 74   1  70]
  [ 74   1  70]
  [ 73   0  69]]

 [[ 75   2  73]
  [ 75   2  73]
  [ 75   2  73]
  ...
  [ 75   2  71]
  [ 75   2  71]
  [ 75   2  71]]

 ...

 [[  1  75 234]
  [  1  75 234]
  [  0  75 229]
  ...
  [ 62  48 145]
  [ 62  48 143]
  [ 61  47 144]]

 [[ 36  49 179]
  [ 36  49 177]
  [ 36  50 173]
  ...
  [ 68  62 152]
  [ 68  61 154]
  [ 67  61 151]]

 [[ 55  65 215]
  [ 54  66 214]
  [ 54  66 214]
  ...
  [ 30 140 255]
  [ 30 142 254]
  [ 29 139 252]]]
<class 'PIL.Image.Image'>
RGB
(1360, 768)
[[ 32  32  32 ...  33  32  32]
 [ 32  32  32 ...  31  31  30]
 [ 32  32  32 ...  32  32  32]
 ...
 [ 71  71  70 ...  63  63  62]
 [ 60  60  60 ...  74  74  73]
 [ 79  79  79 ... 120 121 119]]


Creating a function to load image and convert to greyscale

In [4]:
def imageLoadGrey(image):
    "Converts an image to greyscale and return an array about it"
    return np.array(image.convert("L"));

#testing the function
print(imageLoadGrey(imageTest));

[[ 32  32  32 ...  33  32  32]
 [ 32  32  32 ...  31  31  30]
 [ 32  32  32 ...  32  32  32]
 ...
 [ 71  71  70 ...  63  63  62]
 [ 60  60  60 ...  74  74  73]
 [ 79  79  79 ... 120 121 119]]


The Dataset utilized is Balanced dataset within the Extended MNIST

1. The EMNIST Balanced dataset is meant to address the balance issues in the ByClass and ByMerge datasets. It is derived from the ByMerge dataset to reduce     mis-classification errors due to capital and lower case letters and also has an equal number of samples per class. This dataset is meant to be the most applicable.
    train: 112,800
    test: 18,800
    total: 131,600
    classes: 47 (balanced)

2. CSV  (combined labels and images)
    The datasets are called "emnist-balanced-test" and "emnist-balanced-train" every element is separated by comma and there are no empty elements
    Each row is a separate image.
    The dataset has 785 colums and the First column is the class label (see mappings.txt for class label definitions).
    Each column after the class label, represents one pixel value (784 total for a 28 x 28 image).
    Each pixes has and associated value raging from 0 to 255(inclusive), indicating its greyscale.

In [8]:
#readinf the dataset test and train and showing it's first 5 rows
dataSetTest = pd.read_csv("archive\emnist-balanced-test.csv", sep=",");
dataSetTrain = pd.read_csv("archive\emnist-balanced-train.csv", sep=",");
dataSetTest.head(5);
dataSetTrain.head(5);

In [ ]:
from google.colab import drive
drive.mount("/content/drive");